In [1]:
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen
from snntorch import utils
from snntorch import functional as SF
from snntorch import surrogate

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

import os
import shutil
import matplotlib.pyplot as plt
import numpy as np
import itertools
from tqdm import tqdm
# from collections import defaultdict

from module.custom_data import LoadDataset
from module import custom_data
from module import network, compute_loss, define_net

import matplotlib.pyplot as plt
from IPython.display import HTML

from collections import defaultdict


# いろいろ定義

In [2]:
# Network Architecture
num_inputs = 28*28
num_hidden = 1000
num_outputs = 10
dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# Temporal Dynamics
num_steps = 10
beta = 0.95
dataset_path = "dataset/"
batch_size = 1

train_dataset = LoadDataset(dir = dataset_path, train=True)
test_dataset = LoadDataset(dir = dataset_path,  train=False)

train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=custom_data.custom_collate, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=custom_data.custom_collate, shuffle=False,)



net= define_net.net
model_path = 'models/model1.pth'
net.network.load_state_dict(torch.load(model_path))



optimizer = torch.optim.Adam(net.network.parameters(), lr=5e-4, betas=(0.9, 0.999))
# loss_fn = SF.mse_count_loss(correct_rate=0.8, incorrect_rate=0.2)

num_epochs = 100
num_iters = 50
pixel = 64
correct_rate = 0.5
loss_hist = []



# 出力結果の解析
危険とみなすスパイク数の閾値を変化させたときの、出力画像やiouを算出する


In [3]:


def save_img(pred_pro, label, image_path):
    # label = label.reshape((pixel, pixel)).to('cpu')
    print(pred_pro.shape)
    num_steps = len(pred_pro)
    nrows = 2
    ncols = 5
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(13,8), tight_layout = True)
    th_rate = 0.1
    for i in range(nrows):
        for j in range(ncols):
            if i == 0 and j == 0:
                _label = label.reshape((pixel, pixel)).to('cpu')
                axes[i, j].imshow(_label)
                axes[i, j].set_title('label')
                continue
            # pred_pro = compute_loss.show_pred(pred_pro, th_rate)
            iou = compute_loss.culc_iou(pred_pro, label, th_rate)
            pred_pro_ = pred_pro[:, 1, :, :]
           
            # pred_pro_ = pred_pro[:, 0, :, :]
            # print(pred_pro_.shape)
        
            pred_pro_ = pred_pro_.reshape((pixel, pixel)).to('cpu')
            pred_pro_ = torch.where(pred_pro_>= th_rate, 1, 0)
            axes[i,j].imshow(pred_pro_)
            axes[i, j].set_title(f'pred_pro(th={th_rate})\n IoU:{round(iou,3)}')
            hist[round(th_rate, 1)].append(iou)
            th_rate += 0.1
    plt.tight_layout()
    plt.savefig(image_path)
    plt.close()

hist = defaultdict(list)
result_dir = 'result_img'
if os.path.exists(result_dir):
        shutil.rmtree(result_dir)
os.makedirs(result_dir)

with torch.no_grad():
    net.network.eval()
    for i, (data, label) in enumerate(iter(test_loader)):
        data = data.to(device)
        label = label.to(device)
        batch = len(data[0])
        data = data.reshape(num_steps, batch, 1, pixel, pixel)
        pred_pro = net(data)
        
        iou = compute_loss.culc_iou(pred_pro, label, correct_rate)

        # pred_pro = compute_loss.show_pred(pred_pro, correct_rate)
        img_path = os.path.join(result_dir, f'{str(i).zfill(4)}.png')
      
        save_img(pred_pro, label, img_path)


torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 64, 64])
t

## 閾値毎のIOUの平均算出

In [6]:
for key, value in hist.items():
    print(f'IoU_{key}:{np.mean(value)}')

IoU_0.1:0.7675151172094047
IoU_0.2:0.7658856228366494
IoU_0.3:0.7599267065711319
IoU_0.4:0.7599267065711319
IoU_0.5:0.7599267065711319
IoU_0.6:0.7427386721968651
IoU_0.7:0.7427386721968651
IoU_0.8:0.7220206386968493
IoU_0.9:0.6768500855378806
